# Cortex Analyst徹底検証 #1 セットアップ

## 概要
- このNotebookは、Zennの記事：[Snowflake Cortex Analyst徹底検証 #1：Text-to-SQLの基本と単表/マルチターン精度を上げる設計](https://zenn.dev/nttdata_tech/articles/b7e27f17e348a7)に対応します  
- このNotebookは、上記のZenn記事の環境設定を補助するために使用します。
- 各自のSnowflakeアカウント>Notebooksに、この.ipynbファイルをimportして使用してください
- 以降の項番は、Zennの記事に対応させています

## 3.1 環境準備

### ステップ1.基本設定
DB,SCHEMAを作成します

In [ ]:
-- 検証用のDATABA
USE ROLE SYSADMIN;
CREATE OR REPLACE DATABASE CORTEX_LAB;
CREATE OR REPLACE SCHEMA CORTEX_LAB.L1;
USE SCHEMA CORTEX_LAB.L1;

### ステップ2. テーブルの作成
SALES_L1 テーブルを作成します。

In [ ]:
CREATE OR REPLACE TABLE SALES_L1 (
  ORDER_ID   NUMBER            NOT NULL,
  ORDER_DATE DATE              NOT NULL,
  ITEM_ID    NUMBER            NOT NULL,
  ITEM_NAME  STRING            NOT NULL,
  CATEGORY   STRING            NOT NULL,
  PRICE      NUMBER(10,2)      NOT NULL,
  QUANTITY   NUMBER(4)         NOT NULL
);

### ステップ3. サンプルデータの挿入
SALES_L1にサンプルデータを挿入します。

In [ ]:
INSERT INTO SALES_L1
WITH PARAMS AS (
  SELECT
    ARRAY_CONSTRUCT('Beverage','Snack','Seafood','Meat','Vegetable','Fruit','Dairy','Staple','Sweets','Other') AS CATS,
    ARRAY_CONSTRUCT('Cola','Tea','Coffee','Chips','Nuts','Cookie','Salmon','Tuna','Beef','Pork','Chicken','Lettuce','Tomato','Banana','Milk','Cheese','Rice','Bread','Cake','Chocolate') AS ITEMS
),
BASE AS (
  SELECT SEQ8() AS SEQ
  FROM TABLE(GENERATOR(ROWCOUNT=>3000))
),
RAND_BASE AS (
  SELECT
    SEQ + 1 AS ORDER_ID,
    -- 2023〜2025 を中心にバラける日付（境界は定数なので UNIFORM 可）
    DATEADD(day, UNIFORM(-365, 1095, RANDOM()), DATE('2024-01-01')) AS ORDER_DATE,
    -- 0..19 / 0..9 を明示的に INT にキャスト
    CAST(UNIFORM(0, 20, RANDOM()) AS INT) AS IDX_ITEM,
    CAST(UNIFORM(0, 10, RANDOM()) AS INT) AS IDX_CAT,
    RANDOM() AS R
  FROM BASE
),
CHOICED AS (
  SELECT
    ORDER_ID,
    ORDER_DATE,
    -- バイアス後も INT に揃える
    CAST(CASE WHEN R < 0.30 THEN 0 ELSE IDX_ITEM END AS INT) AS IDX_ITEM_B,
    CAST(CASE WHEN R < 0.10 THEN 0 ELSE IDX_CAT  END AS INT) AS IDX_CAT_B
  FROM RAND_BASE
)
SELECT
  c.ORDER_ID,
  c.ORDER_DATE,
  1000 + c.IDX_ITEM_B                            AS ITEM_ID,
  COALESCE(GET(p.ITEMS, c.IDX_ITEM_B)::STRING, 'UNKNOWN_ITEM') AS ITEM_NAME,
  COALESCE(GET(p.CATS , c.IDX_CAT_B )::STRING, 'Other')        AS CATEGORY,
  -- 100.00〜5000.00（NUMERICに丸め）
  ROUND(UNIFORM(10000, 500000, RANDOM()) / 100.0, 2)           AS PRICE,
  -- 1..5 を UNIFORM で明示的に生成し INT キャスト
  CAST(UNIFORM(1, 6, RANDOM()) AS INT)                         AS QUANTITY
FROM CHOICED c
CROSS JOIN PARAMS p;

In [ ]:
-- テーブルを確認
SELECT * FROM SALES_L1 limit100;

事前準備は以上です。各自、Cortex Analystの設定に移ってください。  

- Snowsight>AI&ML>Cortex Analyst
- ymlファイルはこのNotebookと同じリポジトリに格納しています。